In [1]:
import datetime

import geopandas as gpd
import pandas as pd

import folium
from matplotlib import pyplot as plt
import process as fe
import shapely
from scipy.stats import zscore
import numpy as np

# Interactive Fare Evasion Map

This notebook will demonostrate the creation of a simple `leaflet.js` map representing fare evasion arrests using a data set created in the `Filter Arrests` and the `folium` python package

In [2]:
# Load data using helper functions

stations, evasions, census, lines, nyc = fe.load_all_data()

FileNotFoundError: [Errno 2] No such file or directory: '2013_2017_acs.dta'

Depending on the way the `evasions` DataFrame is made, the date filtering below may not be necessary, but it allows the data in the map to be fine-tuned without changing the saved data set.

In [ ]:
# We only want arrests after 2010
evasions = evasions[evasions['ARREST_DATE'] >= datetime.datetime(2010, 3, 7, 0, 0)]

stations = fe.populate_arrests(stations, evasions)

stations

If you just need a static map, the helper function `plot_num_arrests_station` can take care of that. Passing `save=False` stops the figure from automatically being saved. 

In [ ]:
fe.plot_num_arrests_station(stations, lines, nyc, census, save=False, mult=0.25)

## Making and populating the map 

First, a folium `Map` object is created. All the elements of the map (subway stations, shapes, etc) are added to this map object.

In [ ]:
m = folium.Map(location=(stations.loc['qn009'].geometry.y, stations.loc['qn009'].geometry.x),
               zoom_start=12,
               tiles='CartoDB positron')

I wrote functions to add subway lines and stations to the map such that they accept a single row of data, which allows them to be used in combination with pandas' `DataFrame.apply`. I really enjoy this way of coding since it lets me change the data set and map behavior independently of each other, and only the folium map object changes. It made experimental coding and fine-tuning the parameters a lot easier than it would have been with for loops and row iteration (and I think the code is cleaner as well). 

In [ ]:
# TODO: Document, Move these functions to their own module, called something like `mapmaking.py`

def add_to_map(row, mp, tooltip='Click Here'):
    
    lat = row['stop_lat']
    long = row['stop_lon']
        
#     print(row['arrests'], row['arrests'] / 100)

    popup=folium.Popup(f"<i>{row['complex_nm']}</i><br> has had {int(row['arrests'])} arrests since 2010")


    tooltip = row['complex_nm']
        
    folium.Circle([lat, long], 
                  popup=popup, 
                  color='red',# if arrest_z > 0 else 'blue',
                  radius=row['arrests'] / 8 + 2,
                  weight=1.5,
                  fill=True,
                  tooltip=tooltip).add_to(mp)

In [ ]:
def add_line_to_map(row, mp):
    geom = row.geometry.__geo_interface__
    
    lns = [(y, x) for (x, y) in geom['coordinates']]
    
    folium.PolyLine(lns, color=row['color']).add_to(mp)
    
    return row['color']

Here I just add the official colors for each subway line so that they are easily identifiable on the map

In [ ]:
line_groups = [line for line in lines.groupby('rt_symbol').groups]
colors = ['#EE352E', '#00933C', '#B933AD', '#0039A6', '#FF6319', '#6CBE45', '#996633', '#A7A9AC', '#FCCC0A']

line_colors = {line:color for line, color in zip(line_groups, colors)}

lines['color'] = lines['rt_symbol'].apply(lambda x: line_colors[x])

And finally, here I actually populate the map!

In [ ]:
lines.apply(add_line_to_map, axis=1, args=(m,));

In [ ]:
# Sort by descending arrests so larger circles are placed first, 
# enabling the user to click on every circle even if it inscribes another
st = stations.sort_values(by='arrests', ascending=False)

st.apply(add_to_map, axis=1, args=(m,));

The map can be explored by opening the `index.html` in a browser (recommended) or by calling the jupyter notebook representation of the map object (below).

In [ ]:
m.save('index.html')

In [ ]:
m